In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
import json
from datasets import Dataset, DatasetDict

In [3]:
# Load the BART model and tokenizer
model_name = 'facebook/bart-base'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# Load the preprocessed dataset
with open('preprocessed_data.json', 'r') as f:
    data = json.load(f)

In [5]:
# Prepare the dataset for training and evaluation
train_data = []
eval_data = []
for i, entry in enumerate(data):
    for prompt in entry['prompts']:
        example = {
            'input_text': prompt,
            'target_text': entry['marked_sentence']
        }
        if i % 10 == 0:  # Use 10% of the data for evaluation
            eval_data.append(example)
        else:
            train_data.append(example)

In [6]:
# Convert the training and evaluation data into Dataset objects
datasets = DatasetDict({
    'train': Dataset.from_list(train_data),
    'eval': Dataset.from_list(eval_data)
})

In [7]:
# Tokenize the input and target texts
def tokenize_function(example):
    input_encoding = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    target_encoding = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=512)
    input_encoding['labels'] = target_encoding['input_ids']
    return input_encoding

tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/7371 [00:00<?, ? examples/s]

Map:   0%|          | 0/823 [00:00<?, ? examples/s]

In [8]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    learning_rate=5e-5,
    fp16=True if torch.cuda.is_available() else False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'],
    tokenizer=tokenizer,
)

<ipython-input-9-9f34de2f6631>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_bart')
tokenizer.save_pretrained('./fine_tuned_bart')

print("Model fine-tuning completed and saved to './fine_tuned_bart'")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.002900,0.000913
1000,0.005000,0.000823
1500,0.001000,0.000613
2000,0.001500,0.000541
2500,0.003000,0.000542
3000,0.001300,0.000546
3500,0.000200,0.000481
4000,0.000100,0.000423
4500,0.000400,0.000493
5000,0.000300,0.000451


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model fine-tuning completed and saved to './fine_tuned_bart'


In [36]:
# Evaluate on the evaluation set
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.0004964735126122832, 'eval_runtime': 16.4754, 'eval_samples_per_second': 49.953, 'eval_steps_per_second': 6.252, 'epoch': 3.0}


In [17]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


In [14]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7763b88451aa207c5fb1ab6ed41319a1896a3be2e6d9c29bea8c13f20a090b65
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [15]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline
import json
from datasets import Dataset
import re
from sklearn.metrics import precision_recall_fscore_support
from rouge_score import rouge_scorer

# Load the fine-tuned BART model and tokenizer
model_path = './fine_tuned_bart'
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

# Sample input for testing
sample_data = [
    {
        "input_text": "Extract the details of the Disease_disorder event occurring in this text: A 58-year-old man with poorly controlled hypertension presented to the ICU for cardiogenic shock.",
        "target_text": "A <arg>58-year-old</arg> man with poorly controlled <trigger>hypertension</trigger> presented to the <arg>ICU</arg> for <trigger>cardiogenic shock</trigger>."
    },
    {
        "input_text": "Extract the details of the Clinical_event event occurring in this text: The patient experienced sudden chest pain, indicating a myocardial infarction.",
        "target_text": "The patient experienced sudden <trigger>chest pain</trigger>, indicating a <trigger>myocardial infarction</trigger>."
    }
]

# Convert the sample data into a Dataset object
test_dataset = Dataset.from_list(sample_data)

# Tokenize the input texts
def tokenize_function(example):
    input_encoding = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    target_encoding = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=512)
    input_encoding['labels'] = target_encoding['input_ids']
    return input_encoding

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Define a function to add tags using a simple rule-based approach
def add_tags_post_processing(generated_text, reference_text):
    # Extract all arguments and triggers from the reference text
    arg_pattern = r'<arg>(.*?)</arg>'
    trigger_pattern = r'<trigger>(.*?)</trigger>'

    args = re.findall(arg_pattern, reference_text)
    triggers = re.findall(trigger_pattern, reference_text)

    # Apply tags to generated text using a simple rule-based approach
    for arg in args:
        generated_text = re.sub(rf'\b{re.escape(arg)}\b', f'<arg>{arg}</arg>', generated_text)
    for trigger in triggers:
        generated_text = re.sub(rf'\b{re.escape(trigger)}\b', f'<trigger>{trigger}</trigger>', generated_text)

    return generated_text

# Define the function to test and evaluate the model
def test_and_evaluate_model(model, dataset):
    model.eval()
    generated_texts = []
    references = []

    for i in range(len(dataset)):
        input_text = dataset[i]['input_text']
        target_text = dataset[i]['target_text']

        # Tokenize the input text
        inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512).to(model.device)

        # Generate output using the model
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)

        # Decode the generated output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Apply post-processing to add tags
        generated_text_with_tags = add_tags_post_processing(generated_text, target_text)

        # Append results
        generated_texts.append(generated_text_with_tags)
        references.append(target_text)

        # Print input, reference, and generated output
        print(f"Input: {input_text}")
        print(f"Reference: {target_text}")
        print(f"Generated (without tags): {generated_text}")
        print(f"Generated (with tags): {generated_text_with_tags}")
        print("-" * 50)

    # Evaluate the model using precision, recall, and F1-score
    y_true = []
    y_pred = []
    for ref, gen in zip(references, generated_texts):
        ref_tags = re.findall(r'<(arg|trigger)>.*?</\1>', ref)
        gen_tags = re.findall(r'<(arg|trigger)>.*?</\1>', gen)
        y_true.extend(ref_tags)
        y_pred.extend(gen_tags)

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Evaluate using ROUGE score
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [rouge_scorer_obj.score(ref, gen) for ref, gen in zip(references, generated_texts)]
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

    print(f"Average ROUGE-1 F1 Score: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2 F1 Score: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L F1 Score: {avg_rougeL:.4f}")

# Test and evaluate the model
test_and_evaluate_model(model, tokenized_test_dataset)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Input: Extract the details of the Disease_disorder event occurring in this text: A 58-year-old man with poorly controlled hypertension presented to the ICU for cardiogenic shock.
Reference: A <arg>58-year-old</arg> man with poorly controlled <trigger>hypertension</trigger> presented to the <arg>ICU</arg> for <trigger>cardiogenic shock</trigger>.
Generated (without tags): A 58-year-old man with poorly controlled hypertension presented to the ICU for cardiogenic shock.
Generated (with tags): A <arg>58-year-old</arg> man with poorly controlled <trigger>hypertension</trigger> presented to the <arg>ICU</arg> for <trigger>cardiogenic shock</trigger>.
--------------------------------------------------
Input: Extract the details of the Clinical_event event occurring in this text: The patient experienced sudden chest pain, indicating a myocardial infarction.
Reference: The patient experienced sudden <trigger>chest pain</trigger>, indicating a <trigger>myocardial infarction</trigger>.
Generated 